# Laboratory #07 : Image Classification using Bag of Visual Words

At the end of this laboratory, you would get familiarized with

*   Creating Bag of Visual Words
    *   Feature Extraction
    *   Codebook construction
    *   Classification
*   Using pre-trained deep networks for feature extraction

**Remember this is a graded exercise.**

*   For every plot, make sure you provide appropriate titles, axis labels, legends, wherever applicable.
*   Create reusable functions where ever possible, so that the code could be reused at different places.
*   Use will have to mount your drive if you need to access images.

---

In [1]:
# Loading necessary libraries (Feel free to add new libraries if you need for any computation)

import os
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from skimage.feature import ORB
from skimage.color import rgb2gray
from skimage.io import imread
from skimage import feature

from matplotlib import pyplot as plt

In [2]:
def loadImage(imagePath, gray=False):
    image = imread(imagePath)
    if gray and len(image.shape) > 2:
        image_gray = image
        image_gray = rgb2gray(image_gray)
        return image_gray
    return image

## Loading dataset

We will use 3 categories from Caltech 101 objects dataset for this experiment. Upload the dataset to the drive and mount it.

In [3]:
# modify the dataset variable with the path from your drive

dataset_path = '.'

In [4]:
# creating the list of files and corresponding labels

data = []
labels = []

categories = ['butterfly', 'kangaroo', 'dalmatian']

for idx, cat in enumerate(categories):
    path = os.path.join(dataset_path, cat)
    files_list = os.listdir(path)
    for filename in files_list:
        data.append(os.path.join(path, filename))
        labels.append(idx)

labels = np.array(labels)

print('Total number of images:', len(data))

Total number of images: 244


In [5]:
ncl = len(categories) * 10

In [6]:
# creating train test split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print('Train set:', len(x_train))
print('Test set:', len(x_test))

Train set: 195
Test set: 49


## Feature Extraction using ORB

The first step is to extract descriptors for each image in our dataset. We will use ORB to extract descriptors.

*   Create ORB detector with 64 keypoints.


In [7]:
# solution
orbExtractor = feature.ORB(n_keypoints=64)

*   Extract ORB descriptors from all the images in the train set.
*   Note that, each patch would produce a descriptor of size (256,). So each image would return (N x 256) descriptors, where N is the number of keypoints.

In [8]:
# solution
x_trainOrb = []
for imagePath in tqdm(x_train):
    image = loadImage(imagePath, gray=True)
    orbExtractor.detect_and_extract(image)
    x_trainOrb.append(orbExtractor.descriptors)

100%|██████████| 195/195 [00:41<00:00,  4.66it/s]


In [9]:
# solution
print("number of images: ", len(x_trainOrb))
print("shape of features extracted: ", x_trainOrb[0].shape)
assert x_trainOrb[0].shape == (64, 256)

number of images:  195
shape of features extracted:  (64, 256)


## Codebook Construction

Codewords are nothing but vector representation of similar patches. This codeword produces a codebook similar to a word dictionary. We will create the codebook using K-Means algorithm

*   Create a codebook using K-Means with k=number_of_classes*10
*   Hint: Use sklearn.cluster.MiniBatchKMeans for K-Means

In [10]:
from sklearn.cluster import MiniBatchKMeans

In [11]:
# solution
descriptors = np.vstack(x_trainOrb)
clustering = MiniBatchKMeans(n_clusters=ncl)
klabels = clustering.fit_predict(descriptors)

*   Create a histogram using the cluster centers for each image descriptor.
    *   Remember the histogram would be of size *n_images x n_clusters*.
    *   Calculate the histogram and divide by the feature length to normalize

In [12]:
# solution
klabels_img = klabels.reshape(-1, 64)
cluster_histograms = []
for image_labels in klabels_img:
    hist, _ = np.histogram(image_labels, bins=ncl)
    hist = hist / image_labels.shape[0]
    cluster_histograms.append(hist)
dictionary = np.vstack(cluster_histograms)
dictionary.shape

(195, 30)


# Creating Classification Model

*   The next step is to create a classification model. We will use a C-Support Vector Classification for creating the model.



In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [14]:
# note the usage of grid search to select the best parameters

gammas = np.logspace(-6, -1, 10)
C = np.array([0.5, 1, 2, 4, 8, 10, 15, 20, 50, 100, 200, 375, 500, 1000])

svc = SVC(decision_function_shape='ovr')

clf = GridSearchCV(estimator=svc, param_grid=dict(gamma=gammas, C=C), n_jobs=-1)

clf.fit(dictionary, y_train)

print('Best accuracy:', clf.best_score_) 
print('The best value of gamma:', clf.best_estimator_.gamma)
print('The best value of C:', clf.best_estimator_.C)

Best accuracy: 0.4717948717948718
The best value of gamma: 0.02782559402207126
The best value of C: 200.0


# Testing the Classification Model

*   Extract descriptors using ORB for the test split
*   Use the previously trained k-means to generate the histogram
*   Use the classifier to predict the label


In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [16]:
# solution
x_testOrb = []
for imagePath in tqdm(x_test):
    image = loadImage(imagePath, gray=True)
    orbExtractor.detect_and_extract(image)
    x_testOrb.append(orbExtractor.descriptors)

100%|██████████| 49/49 [00:11<00:00,  4.38it/s]


In [17]:
testklabels = clustering.predict(np.vstack(x_testOrb))

In [18]:
klabels_test_img = testklabels.reshape(-1, 64)
print(klabels_test_img.shape)
test_cluster_histograms = []
for image_labels in klabels_test_img:
    hist, _ = np.histogram(image_labels, bins=ncl)
    hist = hist / image_labels.shape[0]
    test_cluster_histograms.append(hist)
testdictionary = np.vstack(test_cluster_histograms)
testdictionary.shape

(49, 64)


(49, 30)

In [19]:
predicted_labels = clf.predict(testdictionary)

In [20]:
print(predicted_labels.shape)
print(y_test.shape)

(49,)
(49,)


*   Calculate the accuracy score for the classification model

In [21]:
# solution
print(confusion_matrix(predicted_labels, y_test))
print(f"Classification accuracy: {100*accuracy_score(predicted_labels, y_test):.2f}%")


[[12  5  2]
 [ 7 10  8]
 [ 0  3  2]]
Classification accuracy: 48.98%


*   Why do we use Clustering to create the codebook? 
*   What are the other techniques that can be used to create the codebook?

**Solution**

*(Double-click or enter to edit)*

...

# Extracting features from Deep Network

It is quite possible to extract features (similar to SIFT or ORB) from different layers of deep network.

*   Load ResNet50 model with imagenet weights and check the summary of the model
*   Create a model to extract features from the 'avg_pool' layer.
*   Extract features from the layer for all the train images.

In [22]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import Model

In [23]:
# solution
feat_extraction_model = ResNet50(weights='imagenet', include_top=False)

In [49]:
# solution
resNetFeats = []
numFeatures = []
for imagePath in x_train:
    image = loadImage(imagePath)
    if len(image.shape) == 2:
        # convert to rgb (replicate if the image is grayscale)
        image = np.moveaxis(np.array([image, image, image]), [0, 1, 2], [2, 0, 1])
    output = feat_extraction_model(image[np.newaxis, :])
    output = output.numpy().reshape(-1, output.shape[-1])
    resNetFeats.append(output)
    numFeatures.append(output.shape[0])

*   Create codebook using the extracted features

In [54]:
# solution
stackedResNetFeats = np.vstack(resNetFeats)
cumNumFeatures = np.cumsum(numFeatures).astype(np.int)
print(stackedResNetFeats.shape)
print(cumNumFeatures[0], cumNumFeatures[-1], len(cumNumFeatures))

(15610, 2048)
90 15610 195


In [53]:
# solution
clusteringResNetFeats = MiniBatchKMeans(n_clusters=ncl)
resnetkLabels = clusteringResNetFeats.fit_predict(stackedResNetFeats)

In [59]:
# solution
cluster_histograms = []
for idx in range(len(cumNumFeatures)):
    end_idx = cumNumFeatures[idx]
    start_idx = 0 if idx == 0 else cumNumFeatures[idx - 1]

    image_labels = resnetkLabels[start_idx:end_idx]
    # print(start_idx, end_idx, image_labels.shape)
    hist, _ = np.histogram(resnetkLabels[start_idx:end_idx], bins=ncl)
    hist = hist / image_labels.shape[0]
    cluster_histograms.append(hist)
dictionary = np.vstack(cluster_histograms)
dictionary.shape


0 90 (90,)
90 160 (70,)
160 240 (80,)
240 310 (70,)
310 390 (80,)
390 460 (70,)
460 550 (90,)
550 630 (80,)
630 700 (70,)
700 770 (70,)
770 850 (80,)
850 950 (100,)
950 1040 (90,)
1040 1130 (90,)
1130 1220 (90,)
1220 1300 (80,)
1300 1370 (70,)
1370 1440 (70,)
1440 1510 (70,)
1510 1580 (70,)
1580 1650 (70,)
1650 1730 (80,)
1730 1790 (60,)
1790 1860 (70,)
1860 1950 (90,)
1950 2020 (70,)
2020 2110 (90,)
2110 2210 (100,)
2210 2300 (90,)
2300 2390 (90,)
2390 2460 (70,)
2460 2530 (70,)
2530 2610 (80,)
2610 2670 (60,)
2670 2750 (80,)
2750 2830 (80,)
2830 2910 (80,)
2910 2980 (70,)
2980 3060 (80,)
3060 3140 (80,)
3140 3230 (90,)
3230 3300 (70,)
3300 3380 (80,)
3380 3460 (80,)
3460 3530 (70,)
3530 3620 (90,)
3620 3690 (70,)
3690 3780 (90,)
3780 3850 (70,)
3850 3950 (100,)
3950 4050 (100,)
4050 4140 (90,)
4140 4230 (90,)
4230 4300 (70,)
4300 4380 (80,)
4380 4480 (100,)
4480 4540 (60,)
4540 4610 (70,)
4610 4690 (80,)
4690 4770 (80,)
4770 4870 (100,)
4870 4950 (80,)
4950 5020 (70,)
5020 5100 (80,)

(195, 30)

*   Train SVM classifier using the codebook

In [60]:
gammas = np.logspace(-6, -1, 10)
C = np.array([0.5, 1, 2, 4, 8, 10, 15, 20, 50, 100, 200, 375, 500, 1000])

svc = SVC(decision_function_shape='ovr')

clf = GridSearchCV(estimator=svc, param_grid=dict(gamma=gammas, C=C), n_jobs=-1)

clf.fit(dictionary, y_train)

print('Best accuracy:', clf.best_score_) 
print('The best value of gamma:', clf.best_estimator_.gamma)
print('The best value of C:', clf.best_estimator_.C)

Best accuracy: 0.8666666666666668
The best value of gamma: 0.1
The best value of C: 1000.0


*   Evaluate the test set using the above method

In [61]:
# solution
testResNetFeatures = []
testNumFeatures = []
for imagePath in x_test:
    image = loadImage(imagePath)
    if len(image.shape) == 2:
        # convert to rgb (replicate if the image is grayscale)
        image = np.moveaxis(np.array([image, image, image]), [0, 1, 2], [2, 0, 1])
    output = feat_extraction_model(image[np.newaxis, :])
    output = output.numpy().reshape(-1, output.shape[-1])
    testResNetFeatures.append(output)
    testNumFeatures.append(output.shape[0])

In [63]:
testStackedResNetFeats = np.vstack(testResNetFeatures)
testCumNumFeatures = np.cumsum(testNumFeatures).astype(np.int)
print(testStackedResNetFeats.shape)
print(testCumNumFeatures[0], testCumNumFeatures[-1], len(testCumNumFeatures))

(4040, 2048)
80 4040 49


In [64]:
testResnetkLabels = clusteringResNetFeats.predict(testStackedResNetFeats)

In [ ]:
testHistograms = []
for idx in range(len(testCumNumFeatures)):
    end_idx = testCumNumFeatures[idx]
    start_idx = 0 if idx == 0 else testCumNumFeatures[idx - 1]
    image_labels = testResnetkLabels[start_idx:end_idx]
    hist, _ = np.histogram(image_labels, bins=ncl)
    hist = hist / image_labels.shape[0]
    cluster_histograms.append(hist)
dictionary = np.vstack(cluster_histograms)
dictionary.shape

*   Calculate the accuracy score for the classification model

In [ ]:
# solution



## t-distributed Stochastic Neighbor Embedding.

In order to visualize the features of a higher dimension data, t-SNE is used. t-SNE converts the affinities of the data points to probabilities. It recreates the probability distribution in a low-dimensional space. It is very helpful in visualizing features of different layers in a neural network.

You can find more information about t-SNE [here](https://scikit-learn.org/stable/modules/manifold.html#t-distributed-stochastic-neighbor-embedding-t-sne)

In [ ]:
from sklearn.manifold import TSNE

model = TSNE(n_components=2, random_state=0)

np.set_printoptions(suppress=True)

low_embedding = model.fit_transform(dictionary) 

plt.figure(figsize=(20,10))
plt.scatter(low_embedding[:, 0], low_embedding[:, 1], c=y_train)
plt.title("TSNE visualization")
plt.show()

*   What do you infer from the t-SNE plot?

**Solution**

*(Double-click or enter to edit)*

...

*   Compare the performance of both the BoVW models. Which model works better and why?

**Solution**

*(Double-click or enter to edit)*

...

*   Can the performance of pre-trained model increased further? If so, how?

**Solution**

*(Double-click or enter to edit)*

...


---

## **End of P7: Image Classification using Bag of Visual Words**
Deadline for P7 submission in CampusVirtual is: **Thursday, the 26th of November, 2020**